In [1]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, ResetTool, PanTool
from bokeh.models.widgets import Slider, Select, TextInput, Div
from bokeh.models import WheelZoomTool, SaveTool, LassoSelectTool
from bokeh.io import curdoc
from functools import lru_cache


In [2]:
@lru_cache()
def load_data():
    df = pd.read_csv('strava_data.csv', index_col=0)
    return df

In [3]:
run_data_df = load_data()

In [4]:
# all_weeks = list(set(list(load_data()['week'])))
all_weeks = list(load_data()['week'].unique())
X_AXIS = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [5]:
desc = Div(text="Weekly runs", width=800)
weeks_runs = Select(title="Runs", options=all_weeks, value="All")

In [6]:
source = ColumnDataSource(data=load_data())

In [7]:
hover = HoverTool(tooltips=[
    ("Week", "@week"),
    ("Kilometers", "@kms"),
])
TOOLS = [
    hover, BoxZoomTool(), LassoSelectTool(), WheelZoomTool(), PanTool(),
    ResetTool(), SaveTool()
]

In [8]:
p = figure(
    plot_height=600,
    plot_width=700,
    title="Weekly running",
    tools=TOOLS,
    x_axis_label="kms",
    y_axis_label="day od the week",
    toolbar_location="above", 
    x_range=X_AXIS,
    x_minor_ticks=2, y_range=(0, 15),)

p.vbar(x='day_of_week', bottom=0, top='kms', 
         color='blue', width=0.75, 
         legend='Actual', source=source)


GlyphRenderer(id='1046', ...)

In [9]:
def select_weeks():
    """ Use the current selections to determine which filters to apply to the
    data. Return a dataframe of the selected data
    """
    df = load_data()

    # Determine what has been selected for each widgetd
#     cumulative_week_val = weeks_runs.value
    week_val = weeks_runs.value
#     kilometers_val = kms.value

    # Filter by week and weekly_actual_cumulative
    if week_val == "week 01":
        selected = df[df.week == 'week 01'] 
    else:
        selected = df[(df.week == week_val)]

    # Further filter by string in title if it is provided
#     if title_val != "":
#         selected = selected[selected.title.str.contains(title_val, case=False) == True]

#     # Example showing how to update the description
    desc.text = f"Week: {week_val}"
    return selected

In [10]:
def update():
    """ Get the selected data and update the data in the source
    """
    df_active = select_weeks()
    source.data = ColumnDataSource(data=df_active).data

In [11]:
def selection_change(attrname, old, new):
    """ Function will be called when the poly select (or other selection tool)
    is used. Determine which items are selected and show the details below
    the graph
    """
    selected = source.selected["1d"]["indices"]
    
    df_active = select_weeks()
    
    if selected:
        data = df_active.iloc[selected, :]
        temp = data.set_index("week").T.reindex(index=col_order)
        details.text = temp.style.render()
    else:
        details.text = "Selection Details"


    
    

In [12]:

controls = [weeks_runs]

for control in controls:
    control.on_change("value", lambda attr, old, new: update())

source.on_change("selected", selection_change)

inputs = widgetbox(*controls, sizing_mode="fixed")
l = layout([[desc], [weeks_runs, p]], sizing_mode="fixed")

In [13]:
update()
curdoc().add_root(l)
curdoc().title = "Yearly run analysis"

In [14]:
run_data_df.head(10)

,kms,cumulative_daily_actual_kms,daily_goal,cumulative_daily_goal,daily_kms_difference,daily_kms_cumulative_difference,week_number,week,weekly_actual_total,weekly_actual_cumulative,percentage_to_total_weekly,weekly_goal_total,weekly_goal_cumulative,percentage_to_total_weekly_goal,day_of_week,day_number,total_kms,diff_actual_vs_total
run_date,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,5.57,5.57,2.86,2.86,1.95,-14.45,1,week 1.0,32.62,5.57,0.170754,20.02,2.86,0.142857,Mon,1,995.52,27.05
2018-01-02 00:00:00+00:00,5.77,11.34,2.86,5.71,2.02,-8.68,1,week 1.0,32.62,11.34,0.347639,20.02,5.72,0.285714,Tue,2,995.52,21.28
2018-01-03 00:00:00+00:00,4.95,16.29,2.86,8.57,1.73,-3.73,1,week 1.0,32.62,16.29,0.499387,20.02,8.58,0.428571,Wed,3,995.52,16.33
2018-01-04 00:00:00+00:00,0.75,17.04,2.86,11.43,0.26,-2.98,1,week 1.0,32.62,17.04,0.522379,20.02,11.44,0.571429,Thu,4,995.52,15.58
2018-01-05 00:00:00+00:00,5.65,22.69,2.86,14.29,1.98,2.67,1,week 1.0,32.62,22.69,0.695586,20.02,14.30,0.714286,Fri,5,995.52,9.93
2018-01-06 00:00:00+00:00,6.02,28.71,2.86,17.14,2.11,8.69,1,week 1.0,32.62,28.71,0.880135,20.02,17.16,0.857143,Sat,6,995.52,3.91
2018-01-07 00:00:00+00:00,3.91,32.61,2.86,20.00,1.37,12.59,1,week 1.0,32.62,32.62,1.000000,20.02,20.02,1.000000,Sun,7,995.52,0.00
2018-01-08 00:00:00+00:00,3.51,36.12,2.86,22.86,1.23,16.10,2,week 2.0,33.87,3.51,0.103632,20.02,2.86,0.142857,Mon,8,995.52,30.36
2018-01-09 00:00:00+00:00,0.41,36.53,2.86,25.71,0.14,16.51,2,week 2.0,33.87,3.92,0.115737,20.02,5.72,0.285714,Tue,9,995.52,29.95
